# Task 3: Building the RAG Core Logic and Evaluation

In [3]:
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import os
import pickle
import sys
sys.path.append(os.path.abspath("../src"))
import warnings
warnings.filterwarnings('ignore')

In [4]:
# %%
os.chdir("..")  # Go up a directory
#print(os.getcwd())

In [5]:
from RAG_Pipeline import *

In [6]:
if __name__ == "__main__":
    question = "What are common complaints about credit card interest rates?"
    result = rag_pipeline(question)

    # print(" Question:", result["question"])
    print("\n Retrieved Sources (1-2):")
    # for source in result["retrieved_sources"]:
    #     print("-", source['product'], ":", source['original_text'][:200], "...\n")
    # print(" Answer:\n", result["answer"])

Error loading vector store: Error in __cdecl faiss::FileIOReader::FileIOReader(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:68: Error: 'f' failed: could not open FAISS_Index/faiss.index for reading: No such file or directory

 Retrieved Sources (1-2):


In [9]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load your embedding model once
embedder = SentenceTransformer('all-MiniLM-L6-v2')  # replace with your model

# vector_store = {"texts": [...], "embeddings": np.array([...])} - precomputed embeddings and texts

def retrieve_top_k(question, vector_store, top_k=5):
    try:
        q_emb = embedder.encode(question, convert_to_numpy=True)
        embeddings = vector_store['embeddings']
        texts = vector_store['texts']
        
        # cosine similarity
        scores = np.dot(embeddings, q_emb) / (np.linalg.norm(embeddings, axis=1) * np.linalg.norm(q_emb) + 1e-10)
        top_k_indices = np.argsort(scores)[-top_k:][::-1]
        
        results = [(texts[i], float(scores[i])) for i in top_k_indices]
        return results
    except Exception as e:
        print(f"[Retriever error]: {e}")
        return []

In [ ]:
def build_prompt(context_chunks, question):
    context = "\n\n".join(context_chunks)
    prompt = f"""
You are a financial analyst assistant for CrediTrust. Your task is to answer questions about customer complaints. 
Use the following retrieved complaint excerpts to formulate your answer. 
If the context doesn't contain the answer, state that you don't have enough information.

Context:
{context}

Question: {question}

Answer:
"""
    return prompt.strip()

In [11]:
from transformers import pipeline

# Initialize generator once
generator = None
try:
    generator = pipeline("text-generation", model="gpt2", device=-1)  # CPU only; set device=0 for GPU
except Exception as e:
    print(f"[Generator init error]: {e}")

def generate_answer(prompt, max_length=200):
    if generator is None:
        return "Generation model not loaded."
    try:
        outputs = generator(prompt, max_length=max_length, do_sample=True, temperature=0.7)
        return outputs[0]['generated_text'].strip()
    except Exception as e:
        return f"[Generation error]: {e}"


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Error while downloading from https://huggingface.co/gpt2/resolve/main/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...
Device set to use cpu


In [12]:
def rag_pipeline(question, vector_store, top_k=5):
    retrieved = retrieve_top_k(question, vector_store, top_k)
    if not retrieved:
        return {
            "question": question,
            "retrieved_sources": [],
            "answer": "No relevant context found."
        }
    
    retrieved_texts = [txt for txt, score in retrieved]
    prompt = build_prompt(retrieved_texts, question)
    answer = generate_answer(prompt)
    
    return {
        "question": question,
        "retrieved_sources": retrieved[:2],  # returning top 2 with scores
        "answer": answer
    }

In [19]:
import pandas as pd

def qualitative_evaluation(questions, vector_store, top_k=5):
    rows = []
    for q in questions:
        result = rag_pipeline(q, vector_store, top_k)
        retrieved_preview = [txt[:100] + "..." for txt, _ in result['retrieved_sources']]
        rows.append({
            "Question": q,
            "Generated Answer": result['answer'],
            "Retrieved Sources (1-2)": retrieved_preview,
            "Quality Score (1-5)": None,  # to be filled manually
            "Comments/Analysis": ""
        })
    df = pd.DataFrame(rows)
    return df.to_markdown(index=False)

In [21]:
%pip install tabulate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
# Assume vector_store is loaded:
# vector_store = {
#   "texts": [...],  # list of complaint chunks (strings)
#   "embeddings": np.array([...])  # shape (num_chunks, embedding_dim)
# }

questions = [
    "What are common complaints about the mobile app?",
    "How do customers feel about the customer support?",
    # add more representative questions...
]

md_table = qualitative_evaluation(questions, vector_store)
print(md_table)

[Retriever error]: shapes (1,) and (384,) not aligned: 1 (dim 0) != 384 (dim 0)
[Retriever error]: shapes (1,) and (384,) not aligned: 1 (dim 0) != 384 (dim 0)
| Question                                          | Generated Answer           | Retrieved Sources (1-2)   | Quality Score (1-5)   | Comments/Analysis   |
|:--------------------------------------------------|:---------------------------|:--------------------------|:----------------------|:--------------------|
| What are common complaints about the mobile app?  | No relevant context found. | []                        |                       |                     |
| How do customers feel about the customer support? | No relevant context found. | []                        |                       |                     |
